Recherche et extraction des entités nommées avec Transformers.
Modèle de language utilisé: (Camem)BERT.

In [1]:
# @title Installation des bibliothèques, connexion au Google Drive

"""Marina Giardinetti
marina.giardinetti@gmail.com
Google Colab
2024"""

from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
import os
import glob
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

##### Process text sample (from wikipedia)

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
# @title Chargement du corpus
liste_ner =[]

working_dir= '/content/drive/My Drive/AD_L2/' #@param {type:"string"}
corpus = '/content/drive/My Drive/AD_L2/corpus_fem/' #@param {type:"string"}

# Afficher le nombre de textes

textes = os.listdir(corpus)
print('Nombre de textes chargés dans le corpus: ', len(textes))


Nombre de textes chargés dans le corpus:  1


In [6]:
# @title Recherche et extraction des entités nommées
liste_doc=[]
textes = os.listdir(corpus)
for texte in textes:
  content = open(''.join([corpus, texte]))
  content = content.read()
  nom_doc = os.path.basename(texte)

  res = [x for x in re.split("[//.|//!|//?]", content) if x!=""]
  for sentence in res:
    ner = nlp(sentence)
    for dict_ner in ner:
      dict_ner.update({'document':nom_doc})
    liste_ner.append(ner)
    liste_doc.append(nom_doc)
liste_ner2 =[]
for el in liste_ner:
  for el2 in el:
    liste_ner2.append(el2)

df = pd.DataFrame.from_records(liste_ner2)

#@markdown Indiquer le taux de sureté minimal requis pour chaque entité detectée
taux_de_precision = 0.85 #@param {type:"string"}

df_pers = df.loc[(df['entity_group'] == 'PER') & (df['score'] >= taux_de_precision)]
df_pers.to_excel(''.join([working_dir, 'bert_pers.xlsx']))


df_loc = df.loc[(df['entity_group'] == 'LOC') & (df['score'] >= taux_de_precision)]
df_loc.to_excel(''.join([working_dir, 'bert_lieux.xlsx']))
